# OPAN5505 Lab 3 Assignment

## Background

You've been engaged by the Federal Aviation Administration to perform an analysis on "strike" data. A "strike" refers to when an aircraft and an object within the air (typically wildlife) collide.

## Prepare your environment

You will need the `polars` package for this assignment. We'll also use `datetime` for date handling.

In [1]:
# Import required libraries

import polars as pl
from datetime import datetime

print(f"Using Polars version {pl.__version__}")

Using Polars version 1.25.2


## Load your dataset

Read the `faa_strikes.txt` dataset into Polars. Name the resulting DataFrame `strikes`.

In [10]:
# Read the faa_strikes.txt data

strikes = pl.read_csv('/content/faa_strikes.txt', truncate_ragged_lines=True, separator='\t')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Question 1: Running total of strikes by day through 2013

You are interested in seeing the running total of strikes over time (`Collision Date and Time`) on a daily basis. Using the `strikes` dataframe, create a new column that removes the time information but keeps the date information from the `Collision Date and Time` column (call this new field `date`). Next, aggregate the `Number of Strikes` by day (call this new field `daily_strikes`). Sort the data in ascending order by `date`. Then, create the running total of `daily_strikes` (name the new field `strikes_cumulative`) and filter to only the records **up to and including** the `date` of 2013-12-31. Name the resulting data frame `running_total_strikes`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [17]:
# Your code here

running_total_strikes = (
    strikes
    .with_columns(
        date = pl.col('Collision Date and Time').str.slice(0,10).str.strptime(pl.Date, "%Y-%m-%d")
    )
    .group_by('date')
    .agg(
        daily_strikes = pl.col('Number of Strikes').sum()
    )
    .sort('date')
    .with_columns(
        strikes_cumulative = pl.col('daily_strikes').cum_sum().over(pl.col('date'))
    )
    .filter(pl.col('date') <= datetime(2013, 12, 31))
)

display(running_total_strikes)

date,daily_strikes,strikes_cumulative
date,i64,i64
2000-01-02,1,1
2000-01-03,2,2
2000-01-05,3,3
2000-01-06,1,1
2000-01-08,4,4
…,…,…
2013-12-27,6,6
2013-12-28,1,1
2013-12-29,2,2


## Question 2: States with the third highest financial cost

The FAA is interested in the financial cost of strikes for each `Origin State`. Using the `strikes` dataframe: first, sum `Cost: Total $` by `Origin State` (name the resulting column `damage`) then `rank` these states in order of which state had the most financial `damage` (name the new column with rank information, `ranking`). Next, filter to show the row with the third highest `ranking`. Use a window function to answer this question. Name the resulting dataframe `damage_state`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [20]:
# Your code here

damage_state = (
    strikes
    .group_by('Origin State')
    .agg(
        damage = pl.col('Cost: Total $').sum()
    )
    .with_columns(
        dummy_key = pl.lit(1) # Add a dummy column for partitioning
    )
    .with_columns(
        ranking = pl.col('damage').rank(method='min', descending=True).over(['dummy_key']) # Partition by the dummy column
    )
    .filter(pl.col('ranking') == 3)
    .drop('dummy_key') # Drop the dummy column
)

display(damage_state)

Origin State,damage,ranking
str,i64,u32
"""California""",29671432,3


## Question 3: What are the second costliest Species Groups for each Aircraft Type?

The FAA wants to know if some species groups are more dangerous to particular types of aircraft. Using the `strikes` dataframe: first, sum financial damage (`Cost: Total $`) information by `Aircraft: Type` and `Wildlife: Species Group` (name the new field `damage`). Rank the rows within each `Aircraft: Type` based on the greatest amount of `damage`; when performing the ranking function, name the new column `ranking`. Return the rows that represent the `Wildlife: Species Group` values that caused the **second most** financial `damage` to each `Aircraft: Type`. Name the resulting dataframe `type_species`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a ranking function to receive credit on this question.*

In [22]:
# Your code here

type_species = (
    strikes
    .group_by(['Aircraft: Type', 'Wildlife: Species Group'])
    .agg(
        damage = pl.col('Cost: Total $').sum()
    )
    .with_columns(
        ranking = pl.col('damage').rank(method='min', descending=True).over(['Aircraft: Type'])
    )
    .filter(pl.col('ranking') == 2)
)

display(type_species)

Aircraft: Type,Wildlife: Species Group,damage,ranking
str,str,i64,u32
"""NA""","""Vultures""",56227,2
"""Airplane""","""Kites, Hawks, Eagles""",48989287,2
"""Helicopter""","""Vultures""",116748,2


## Question 4: Which days had the greatest positive jump in strikes?

The FAA wants to investigate which days had the largest increase in strikes from the previous day. Using the `strikes` dataframe: sum the `Number of Strikes` measure by day (note: not by day and time). You can use a similar technique from Question 1 to create a new column from `Collision Date and Time` without time information and with date information (name this column `date`). Next, compute the previous day's strikes using a window function (name the new column `previous_day`), and calculate the difference of strikes between the current day and the previous day (name the new column `delta_strikes`). Sort the resulting data by `delta_strikes` in descending order to find which days had the highest increase of strikes from the previous day. Name the resulting dataframe `greatest_strike_increase`.

**Hint:** This code is more involved that other exercises, so I will explicitly name the steps to answer the question.

1. Start with the `strikes` data frame
2. Create a new column called `date` which removes the time information in the `Collision Date and Time` column
3. Sum the `Number of Strikes` by `date`, and name the new column `daily_strikes`
4. Sort by the `date` column in ascending order
5. Create a new column called `previous_day` to calculate what the `daily_strikes` were in the previous day
6. Create a new column called `delta_strikes` which subtracts `previous_day` from `daily_strikes`
7. Sort in descending order by `delta_strikes`

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [24]:
# Your code here

daily_strikes = (
    strikes
    .with_columns(
        date = pl.col('Collision Date and Time').str.slice(0,10).str.strptime(pl.Date, "%Y-%m-%d")
    )
    .group_by('date')
    .agg(
        daily_strikes = pl.col('Number of Strikes').sum()
    )
    .sort('date') # Sort by date before calculating the difference
)


greatest_strike_increase = (
    daily_strikes
    .with_columns(
        previous_day = pl.col('daily_strikes').shift(1).fill_null(0) # Calculate previous day strikes and fill nulls with 0
    )
    .with_columns(
        delta_strikes = pl.col('daily_strikes') - pl.col('previous_day')
    )
    .sort('delta_strikes', descending=True)
)

display(greatest_strike_increase)

date,daily_strikes,previous_day,delta_strikes
date,i64,i64,i64
2010-10-29,25,7,18
2014-08-25,25,8,17
2012-06-28,21,5,16
2009-07-07,21,7,14
2013-11-04,22,8,14
…,…,…,…
2012-06-29,6,21,-15
2012-10-14,5,20,-15
2014-08-26,10,25,-15


## Question 5: Which single day had the greatest increase in strikes for each `Aircraft: Type`?

FAA was interested in the exercise from the last question but now wants to determine the largest delta between days for each `Aircraft: Type`. What are the days that had largest positive change in strikes for each `Aircraft: Type`? In order to answer this question, you will need to perform the exercise from Question 4, but this time include a grouping by `Aircraft: Type`. After grouping by `Aircraft: Type`, add a column called `ranking` and use a window function to determine the day with the largest increase in strikes from the previous day. Name the resulting data frame `greatest_strike_increase_type`

NOTE: Helicopters are not struck with a high frequency and will not show up in your analysis; this is fine.

In [27]:
# Your code here

greatest_strike_increase_type = (
    strikes
    .with_columns(
        date = pl.col('Collision Date and Time').str.slice(0,10).str.strptime(pl.Date, "%Y-%m-%d")
    )
    .group_by(['Aircraft: Type', 'date'])
    .agg(
        daily_strikes = pl.col('Number of Strikes').sum()
    )
    .sort(['Aircraft: Type', 'date'])
    .with_columns(
        previous_day = pl.col('daily_strikes').shift(1).over(['Aircraft: Type']).fill_null(0)
    )
    .with_columns(
        delta_strikes = pl.col('daily_strikes') - pl.col('previous_day')
    )
    .with_columns(
        ranking = pl.col('delta_strikes').rank(method='max', descending=True).over(['Aircraft: Type'])
    )
    .filter(pl.col('ranking') == 1)
)

display(greatest_strike_increase_type)

Aircraft: Type,date,daily_strikes,previous_day,delta_strikes,ranking
str,date,i64,i64,i64,u32
"""Airplane""",2014-08-25,25,8,17,1
"""NA""",2014-08-04,5,1,4,1


In [7]:
print(strikes.columns)

['Airport: Code\tAirport: Name\tOrigin State\tOrigin State Code\tCountry\tAircraft: Type\tAircraft: Number of engines\tCollision Date and Time\tWhen: Time of day\tWhen: Phase of flight\tEffect: Amount of damage (detailed)\tEffect: Impact to flight\tEffect: Indicated Damage\tCost: Aircraft time out of service (hours)\tCost: Total $\tDays\tFeet above ground\tMiles from airport\tWildlife: Animal Category\tWildlife: Species Order\tWildlife: Species Group\tWildlife: Species\tWildlife: Species ID\tNumber of Strikes\tRecord ID']
